### What is an adopted user?

Someone who logged in on three seperate days during a 7-day period. Goal is to indentify which factors predict future user adoption

In [5]:
# Set up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Wrangling

In [7]:
# Load the data
df_engagement = pd.read_csv('takehome_user_engagement.csv')

In [11]:
df_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
